* The prediction parquet to be evaluated needs to contain trackId, model_output, y_true columns.

In [209]:
import org.apache.spark.ml.feature.SQLTransformer
import org.apache.spark.mllib.tree.loss.LogLoss

val predictions_dir = 
val model_name = "date_predictions.parquet/*"
val ddf = spark.read.parquet(s"${predictions_dir}${model_name}")

val formatted_ddf = ddf.selectExpr("target as y_true"," cast(split(split(CAST(probability as STRING), '[,]')[1], ']')[0] as float) as model_output")

formatted_ddf: org.apache.spark.sql.DataFrame = [y_true: int, model_output: float]

In [210]:
val predictions_ddf = formatted_ddf.selectExpr("cast(y_true as double) y_true", 
                                    "cast(model_output as double) model_output",
                                    "cast(model_output>0.5 as double) predicted_class")

predictions_ddf: org.apache.spark.sql.DataFrame = [y_true: double, model_output: double ... 1 more field]

In [211]:
predictions_ddf.describe().show()

+-------+------------------+--------------------+-------------------+
|summary|            y_true|        model_output|    predicted_class|
+-------+------------------+--------------------+-------------------+
|  count|              1019|                1019|               1019|
|   mean|0.2561334641805692| 0.24639845547654152|0.21786064769381747|
| stddev|0.4367107428676595| 0.32786620034097463|0.41299487932146206|
|    min|               0.0|2.842412504833191...|                0.0|
|    max|               1.0|   0.997983992099762|                1.0|
+-------+------------------+--------------------+-------------------+

In [212]:
import org.apache.spark.mllib.evaluation.BinaryClassificationMetrics
import org.apache.spark.mllib.regression.LabeledPoint
import org.apache.spark.mllib.util.MLUtils
import org.apache.spark.mllib.evaluation.MulticlassMetrics

val predictionAndLabels = predictions_ddf.select("model_output", "y_true").rdd.map(row => 
  (row.getAs[Double](0), row.getAs[Double](1)))
val metrics = new BinaryClassificationMetrics(predictionAndLabels)

val predictionAndClasses = predictions_ddf.select("predicted_class", "y_true").rdd.map(row => 
  (row.getAs[Double](0), row.getAs[Double](1)))
val metricsMulti = new MulticlassMetrics(predictionAndClasses)

metricsMulti: org.apache.spark.mllib.evaluation.MulticlassMetrics = org.apache.spark.mllib.evaluation.MulticlassMetrics@3c359eff

In [213]:
println(s"Accuracy = ${metricsMulti.accuracy}")

Accuracy = 0.8694798822374877

In [214]:
println(s"F1 score: ${metricsMulti.weightedFMeasure}")

F1 score: 0.8658471839467836

In [215]:
println("Area under ROC = " + metrics.areaUnderROC)

Area under ROC = 0.9493575551714029